# Cегментации опухоли головного мозга и ее компонентов на основе мультимодальных магнитно-резонансных изображений (МРТ)

In [2]:
import itk

Dimension = 3

FloatPixelType = itk.F

ImageType = itk.Image[FloatPixelType, Dimension]

file_name = '.\\data\\test.mha'
processed_file_name = 'mask.mha'

ReaderType = itk.ImageFileReader[ImageType]
reader = ReaderType.New()
reader.SetFileName(file_name)

itk_image = reader.GetOutput()
print(type(itk_image))

<class 'itk.itkImagePython.itkImageF3'>


# Filtering

## Rescale an Image
### [tutorial link](https://examples.itk.org/src/filtering/imageintensity/rescaleintensity/documentation)

In [3]:
import itk

rescaleIntensityImageFilter = itk.RescaleIntensityImageFilter(
    Input=reader.GetOutput(), 
    OutputMinimum=0, 
    OutputMaximum=255
)


## Threshold an Image Using Binary Thresholding

### [tutorial](https://examples.itk.org/src/filtering/thresholding/thresholdanimageusingbinary/documentation)

In [4]:
import itk

thresholdFilter = itk.ThresholdImageFilter.New(Input=rescaleIntensityImageFilter, Lower=90)

print(type(thresholdFilter))

<class 'itk.itkThresholdImageFilterPython.itkThresholdImageFilterIF3'>


# Segmentation

[tutorial link](https://examples.itk.org/src/segmentation/watersheds/segmentwithwatershedimagefilter/documentation)

In [5]:
import itk

gradientFilter = itk.GradientMagnitudeRecursiveGaussianImageFilter[ImageType, ImageType].New()
gradientFilter.SetSigma(1.0)
gradientFilter.SetInput(thresholdFilter.GetOutput())
gradientFilter.Update()

watershed = itk.WatershedImageFilter.New(Input=gradientFilter.GetOutput())
watershed.SetThreshold(0.001)
watershed.SetLevel(0.25)
watershed.Update()

PixelType = itk.UC
RGBPixelType = itk.RGBPixel[PixelType]
RGBImageType = itk.Image[RGBPixelType, Dimension]

LabeledImageType = type(watershed.GetOutput())

ScalarToRGBColormapImageFilterType = itk.ScalarToRGBColormapImageFilter[
    LabeledImageType, RGBImageType
]

colormapImageFilter = ScalarToRGBColormapImageFilterType.New()
colormapImageFilter.SetColormap(
    itk.ScalarToRGBColormapImageFilterEnums.RGBColormapFilter_Hot
)
colormapImageFilter.SetInput(watershed.GetOutput())
colormapImageFilter.Update()

print(type(colormapImageFilter))

<class 'itk.itkScalarToRGBColormapImageFilterPython.itkScalarToRGBColormapImageFilterIULL3IRGBUC3'>


# Writing

In [6]:
import itk

writer = itk.ImageFileWriter[RGBImageType].New()
writer.SetFileName(processed_file_name)
writer.SetInput(colormapImageFilter.GetOutput())
# writer.SetUseCompression(True)
writer.Update()

# Reading

In [7]:
import itk, vtk

original_reader = vtk.vtkMetaImageReader()
original_reader.SetFileName(file_name)
original_reader.Update()

processed_reader = vtk.vtkMetaImageReader()
processed_reader.SetFileName(processed_file_name)
processed_reader.Update()


# Output

In [26]:
import vtk


mapper = vtk.vtkGPUVolumeRayCastMapper()
mapper.SetInputData(original_reader.GetOutput())

volume = vtk.vtkVolume()
volume.SetMapper(mapper)

data_min, data_max = original_reader.GetOutput().GetScalarRange()

segment_min, seg_max = data_min, 0.6 * data_max

volume_property = volume.GetProperty()

# original
ctf_default = vtk.vtkColorTransferFunction()
ctf_default.AddRGBSegment(segment_min, *(0, 0, 0), seg_max, *(1, 1, 1))
volume_property.SetColor(ctf_default)

opacity_default = vtk.vtkPiecewiseFunction()
opacity_default.AddSegment(segment_min, 0., seg_max, 0.2)
volume_property.SetScalarOpacity(opacity_default)

# processed
ctf_processed = vtk.vtkColorTransferFunction()
ctf_processed.AddRGBSegment(segment_min, *(0, 1, 0), seg_max, *(0, 1, 1))
volume_property.SetLabelColor(1, ctf_processed)

opacity_processed = vtk.vtkPiecewiseFunction()
opacity_processed.AddSegment(segment_min, 0., seg_max, 1.)
volume_property.SetLabelScalarOpacity(1, opacity_processed)

# mask setting
volume_mapper = volume.GetMapper()
volume_mapper.SetMaskInput(processed_reader.GetOutput())


# RENDERING
renderer = vtk.vtkRenderer()
renderer.AddVolume(volume)

renderer_window = vtk.vtkRenderWindow()
renderer_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renderer_window)

interactor.Initialize()
renderer_window.Render()
interactor.Start()

# Custom output

In [9]:
# import vtk

# vtk_image = processed_reader.GetOutput() #itk.vtk_image_from_image(thresholdFilter.GetOutput())
# mapper = vtk.vtkGPUVolumeRayCastMapper()
# mapper.SetInputData(vtk_image)
# # mapper.SetInputConnection(vtk_image)

# volume = vtk.vtkVolume()
# volume.SetMapper(mapper)

# black = (0, 0, 0)
# white = (1, 1, 1)
# green = (0, 1, 0)
# blue  = (0, 0, 1)

# colorDefault = vtk.vtkColorTransferFunction()
# colorDefault.AddRGBPoint(0, *black)
# colorDefault.AddRGBPoint(1, *green)
# colorDefault.AddRGBPoint(2, *white)
# colorDefault.AddRGBPoint(3, *blue)
# colorDefault.AddRGBSegment(4, *black, 255, *white)

# opacityDefault = vtk.vtkPiecewiseFunction()
# opacityDefault.AddPoint(0,0)
# opacityDefault.AddPoint(1,0.1)
# opacityDefault.AddPoint(2,0)
# opacityDefault.AddPoint(3,0)
# opacityDefault.AddSegment(4, 0.,255, 0.01)


# volumeProperty = volume.GetProperty()
# volumeProperty.SetColor(colorDefault)
# volumeProperty.SetScalarOpacity(opacityDefault)
# volumeProperty.SetInterpolationTypeToNearest()

# # --------------------------------------------------
# renderer = vtk.vtkRenderer()
# renderer.AddVolume(volume)

# renderer_window = vtk.vtkRenderWindow()
# renderer_window.AddRenderer(renderer)

# interactor = vtk.vtkRenderWindowInteractor()
# interactor.SetRenderWindow(renderer_window)

# interactor.Initialize()
# interactor.Start()

In [10]:
from itkwidgets import view

view(thresholdFilter)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [11]:
from itkwidgets import view

view(colormapImageFilter)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageRGBUC3;…